# OSI Lat Long
### This Notebook uses the XY of the point featureclass and adds Lat Long to each related row in a related table
A CSV file is generated that will be used to support a seperate project.

The second half of the Notebook explores the electric consumption data with a [SpatialDataFrame](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html?highlight=spatialdataframe#spatialdataframe) using the customers table modifed in the first half.

In [ ]:
import arcpy
import numpy
import pandas as pd

In [ ]:
gdb = r"C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb"
svcPntSource = r"C:\GISData\Data\Snapshot\mxElectric.geodatabase\Electric\main.eServicePoint"
#custAcctSource = r"C:\GISData\Data\Snapshot\mxElectric.geodatabase\main.eCUSTOMERACCOUNT"
svcPntDest = r"C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\eServicePoint"
custAcctDest = r"C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\OSILatLong.gdb\eCUSTOMERACCOUNT"
svcPntDestFlds = ["OID@","POINT_X","POINT_Y","PHASEDESIGNATION"]
custAcctDestFlds = ["OID@","POINT_X","POINT_Y","PHASEDESIGNATION","SERVICEPOINTOBJECTID"]
custAcctOutFlds = ["OID@","SERVICEPOINTOBJECTID","INSTALL_NUM","POINT_X","POINT_Y","PHASEDESIGNATION"]
custAnalysis = ["SERVICEPOINTOBJECTID","INSTALL_NUM","POINT_X","POINT_Y","AVGCONSUMPTION","MAXCONSUMPTION"]
custAcctFile = r'C:\Users\friendde\Documents\ArcGIS\Projects\OSILatLong\GIS.csv'

#### Phase Translation between ArcFM and OSI
| Phase | ArcFM | OSI |
|-------|-------|-----|
|   A   |   4   |  1  ||
|   B   |   2   |  2  |
|   C   |   1   |  3  |
|   AB  |   6   |  12 |
|   AC  |   5   |  13 |
|   BC  |   3   |  23 |
|   ABC |   7   |  123|

In [ ]:
def getPhaseDesignation(phaseDesignation):
        if phaseDesignation is None:
                ph = 0
                return ph
        if phaseDesignation == 1:
                ph = 3
                return ph
        if phaseDesignation == 2:
                ph = 2
                return ph
        if phaseDesignation == 3:
                ph = 23
                return ph
        if phaseDesignation == 4:
                ph = 1
                return ph
        if phaseDesignation == 5:
                ph = 13
                return ph
        if phaseDesignation == 6:
                ph = 12
                return ph
        if phaseDesignation == 7:
                ph = 123
                return ph

In [ ]:
arcpy.env.workspace = gdb
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4326)

In [ ]:
if arcpy.Exists(svcPntDest):
    print(f'Found {svcPntDest}')
    arcpy.Delete_management (svcPntDest)
print(f'Copying {svcPntSource} to {svcPntDest}')
arcpy.Copy_management(svcPntSource,svcPntDest)
if arcpy.Exists(svcPntDest):
    print(f'Copy Success, adding XY')
    arcpy.AddXY_management(svcPntDest)
    print(f'AddXY completed')

In [ ]:
if arcpy.Exists(custAcctDest):
    print(f'Copy Success {custAcctDest}')
    arcpy.AddField_management(custAcctDest, "POINT_X", "DOUBLE")
    arcpy.AddField_management(custAcctDest, "POINT_Y", "DOUBLE")
    arcpy.AddField_management(custAcctDest, "PHASEDESIGNATION", "LONG")
else:
    print(f'Copy Failed! {custAcctDest}')

In [ ]:
edit = arcpy.da.Editor(gdb)
edit.startEditing(False, False)
edit.startOperation()

In [ ]:
with arcpy.da.SearchCursor(svcPntDest,svcPntDestFlds) as svcpnts:
        for svcpnt in svcpnts:
                whereClause = f'SERVICEPOINTOBJECTID = {svcpnt[0]}'
                #whereClause = "SERVICEPOINTOBJECTID = {}".format(svcpnt[0])
                #print(whereClause,svcpnt[1])
                with arcpy.da.UpdateCursor(custAcctDest,custAcctDestFlds,whereClause) as uc:
                    #print(row)
                    for row in uc:
                            row[1] = svcpnt[1]
                            row[2] = svcpnt[2]
                            row[3] = getPhaseDesignation(svcpnt[3])
                            uc.updateRow(row)

In [ ]:
edit.stopOperation()
edit.stopEditing(True)

### Use numpy and pandas to export to CSV

Use arcpy [```TableToNumPyArray()```](http://pro.arcgis.com/en/pro-app/arcpy/data-access/tabletonumpyarray.htm)
See also [Working with numpy in ArcGIS](http://pro.arcgis.com/en/pro-app/arcpy/get-started/working-with-numpy-in-arcgis.htm)

In [ ]:
nparr = arcpy.da.TableToNumPyArray(custAcctDest,custAcctOutFlds,skip_nulls=True)
#nparr = arcpy.da.TableToNumPyArray(custAcctDest,custAcctOutFlds,null_value=-9999)
pdarr = pd.DataFrame(nparr)
pdarr.to_csv(custAcctFile,header=False, index=False)

TODO - write file to \\gruadmin.gru.com\fs\Groups\OMS Replacement Project\Documents for OSII\Customer and Premise Files

Ready new numpy array for consumption analysis

In [ ]:
nparr = arcpy.da.TableToNumPyArray(custAcctDest,["SERVICEPOINTOBJECTID","POINT_X","POINT_Y","AVGCONSUMPTION","MAXCONSUMPTION"],skip_nulls=True)

In [ ]:
df = pd.DataFrame(nparr)
df.head(5)

[```gis.features.SpatialDataFrame()```](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html?highlight=spatialdataframe#arcgis.features.SpatialDataFrame.from_xy)

In [ ]:
from arcgis.features import SpatialDataFrame
from arcgis.gis import GIS
from getpass import getpass
from IPython.display import display

In [ ]:
sdf = SpatialDataFrame.from_xy(df,"POINT_X","POINT_Y")
gis = GIS(arcpy.GetActivePortalURL(), username=input("Enter User Name "), password=(getpass()))
#gis = GIS()
#portalDesc = arcpy.GetPortalDescription()
# search and list all items owned by connected user
#query=f'owner:{portalDesc["user"]["username"]} AND title:CW BaseMap'
#itemType="Feature Layer"
#sortField="title"
#sortOrder="asc"
# default max__items is 10
#maxItems=100
#m = gis.content.search(query,itemType,sortField,sortOrder,maxItems)

In [ ]:
m = gis.map('Gainesville,FL')
m
consumptionLyr = gis.content.import_data(sdf)
m.add_layer(consumptionLyr,options={"renderer":"GraduatedSymbolsRenderer","classificationField":"MAXCONSUMPtION"})

In [ ]:
m